# Digital Earth Canada demonstration - BigWeather Use Case

## Build and upload Docker image
### Get access token

In [ ]:
from copy import deepcopy
import json
import os

from base64 import b64decode
import boto3
import docker
from dotenv import load_dotenv
from IPython import display
import requests
import yaml

In [ ]:
load_dotenv('../default.env')

token_url = f"https://keycloak.{os.getenv('DOMAIN')}/realms/master/protocol/openid-connect/token"

data = {
    'grant_type': 'password',
    'username': os.getenv('USERNAME'),
    'password': os.getenv('PASSWORD'),
    'client_id': os.getenv('CLIENT_ID'),
    'client_secret': os.getenv('CLIENT_SECRET')
}

In [ ]:
response = requests.post(token_url, data=data).json()

In [ ]:
token = response['access_token']

### Get Workspace Information

In [ ]:
workspace_url = f"https://workspace-api.{os.getenv('DOMAIN')}/workspaces/{os.getenv('WORKSPACE')}"

headers = {
    'Authorization': f"Bearer {token}"
}

In [ ]:
response = requests.get(workspace_url, headers=headers).json()
response

In [ ]:
cr_username = response['container_registry']['username']
cr_password = response['container_registry']['password']
cr_registry = response['container_registry']['url']

storage_info = deepcopy(response['storage'])

### Links
https://harbor.dec.alpha.canada.ca/harbor/projects
https://applicationhub.dec.alpha.canada.ca/user/bigweather/?folder=/workspace
https://argo-workflow.dec.alpha.canada.ca/workflows/ws-bigweather-job?&limit=50

In [ ]:
docker_client = docker.DockerClient(base_url=os.getenv('DOCKER_SOCK'))

In [ ]:
docker_client.login(cr_username, cr_password, registry=cr_registry)

In [ ]:
app_tag = f'{cr_registry.replace("https://", "")}/test-tom/precipitations2:0.0.10'
docker_client.images.build(tag=app_tag, path='../app/precipitations')
docker_client.images.push(app_tag)

In [ ]:
with open('../app/precipitations/workflow/workflow_template.yaml') as fh:
    cwl_yaml_dict = yaml.load(fh, Loader=yaml.SafeLoader)
    process_identifier = cwl_yaml_dict['metadata']['name']
    fh.seek(0)
    cwl_yaml_str = fh.read()

url = f"https://zoo.{os.getenv('DOMAIN')}/{os.getenv('USERNAME')}/ogc-api/processes/{process_identifier}"

headers = {
    'Content-Type': 'application/cwl+yaml',
    'Accept': 'application/json',
    'Authorization': f'Bearer {token}'
}

# do not deploy, update
response = requests.put(url, headers=headers, data=cwl_yaml_str)
response.raise_for_status()

In [ ]:
response

In [ ]:
url = f"https://zoo.{os.getenv('DOMAIN')}/{os.getenv('USERNAME')}/ogc-api/processes/{process_identifier}/execution"

headers = {
    'Prefer': 'respond-async;return=representation',
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {token}'
}

data = {
    'inputs': {
        'x': "-123.116",
        'y': "49.288"
    }
}

response = requests.post(url, headers=headers, data=json.dumps(data))
response.raise_for_status()

job_url = response.headers['location']

In [ ]:
headers = {
    'Authorization': f'Bearer {token}'
}

job_poll_end = False

while not job_poll_end:
    response = requests.get(job_url, headers=headers).json()
    if response['status'] == 'successful':
        job_poll_end = True
    elif response['status'] == 'failed':
        print("process failed: {response['message']}")
        job_poll_end = True
    else:
        print(response['status'], response['progress'])


In [ ]:
url = response['links'][0]['href']

response
headers = {
    'Authorization': f'Bearer {token}'
}

response = requests.get(url, headers=headers).json()

In [ ]:
url = response['links'][1]['href']

headers = {
    'Authorization': f'Bearer {token}'
}

response = requests.get(url, headers=headers).json()

In [ ]:
result = json.loads(response['stac']['value'])
result

In [ ]:
url = 's3://ws-bigweather/processing-results/7d95ce4c-0b61-11f0-9e3d-9a8ccd65f1f9/collection.json'
url = result['links'][2]['href']

def get_s3_object_only(s3_url: str):
    return s3_url.replace(f"s3://{storage_info['credentials']['bucketname']}", '')

s3_url = get_s3_object_only(url)

headers = {
    'Authorization': f'Bearer {token}'
}

import boto3
session = boto3.Session(
    aws_access_key_id=storage_info['credentials']['access'],
    aws_secret_access_key=storage_info['credentials']['secret']
)

s3_client = session.client('s3', endpoint_url=storage_info['credentials']['endpoint'])
data = s3_client.get_object(Bucket=storage_info['credentials']['bucketname'], Key=s3_url)

data2 = data['Body'].read()
feature = json.loads(data2)

#print(feature)
probability_of_rain_url = get_s3_object_only(feature['assets']['probability_of_rain']['href'])
predicted_profit_url = get_s3_object_only(feature['assets']['predicted_profit']['href'])

probability_of_rain_data = s3_client.get_object(Bucket=storage_info['credentials']['bucketname'], Key=probability_of_rain_url)['Body'].read()
predicted_profit_data = s3_client.get_object(Bucket=storage_info['credentials']['bucketname'], Key=predicted_profit_url)['Body'].read()

display.Image(predicted_profit_data)


In [ ]:
display.Image(probability_of_rain_data)